# Practical Factor Investing: Dimensional, Avantis, and Portfolio Construction

## From Theory to Practice — Analysing Real Factor ETFs with the Tools We've Built

---

### Where This Fits

This is the fifth notebook in the series. The recommended reading order is:

1. **[Statistical Foundations](01_Statistical_Foundations.ipynb)** — CLT, hypothesis testing, OLS, sandwich estimators, Newey-West derivation
2. **[Time Series Foundations](02_Time_Series_Foundations.ipynb)** — Stationarity, autocorrelation, volatility clustering, ergodicity
3. **[Fama-French 3-Factor Model](03_Fama_French_3Factor.ipynb)** — Data, regression mechanics, diagnostics, robust SEs, interpretation
4. **[Advanced Factor Models](04_Advanced_Factor_Models.ipynb)** — FF5, momentum, profitability, beta anomaly, Chen's challenge, rolling windows
5. **Practical Factor Investing** — this notebook

The previous notebooks gave us the *theory*, the *statistical tools*, and the *research landscape*. This notebook asks: **how do real-world fund managers translate factor research into investable products — and can we verify their claims using the methods we've learned?**

### What We'll Cover

1. **The Dimensional and Avantis Investment Philosophies** — How these firms implement academic factor research, their scientific approach, and what distinguishes them from traditional index funds
2. **Data: Avantis UCITS ETFs on Xetra** — Loading daily return data for AVWS, AVWC, and AVEM alongside global Fama-French factors
3. **Descriptive Analysis** — Return distributions, volatility, drawdowns, and how these ETFs compare
4. **Factor Regression Analysis** — Applying FF5 + Momentum regressions to identify each ETF's factor exposures, with full diagnostic checks
5. **Comparative Factor Profiles** — Visualising how the ETFs differ in their factor tilts and what that implies
6. **Portfolio Construction** — Building a 50% AVWS / 40% AVWC / 10% AVEM portfolio and analysing its risk-return characteristics
7. **Portfolio Factor Analysis** — Running factor regressions on the combined portfolio
8. **Limitations and Conclusions** — What we can and cannot conclude from limited data

### A Note on Data Limitations

> **Important caveat (March 2026):** The Avantis UCITS ETFs launched on European exchanges in late 2024. We have approximately **17 months of daily data** for AVWS and AVWC, and roughly **15 months** for AVEM. This is a *very* short sample for drawing definitive conclusions about factor exposures and alpha.
>
> What we **can** do with daily data:
> - Identify factor **tilts** (betas) with reasonable statistical precision — 300+ daily observations gives enough power for point estimates
> - Characterise the return distribution and risk profile
> - Illustrate how the analytical tools from earlier notebooks apply to real portfolios
>
> What we **cannot** do reliably:
> - Estimate alpha with tight confidence intervals
> - Draw conclusions about long-term performance
> - Compare realised premia to historical averages (the sample is too short)
>
> This notebook is therefore as much a **methodology demonstration** as a performance evaluation. The tools we apply here will remain valid as the data history grows.

### Prerequisites

All four preceding notebooks. We use:
- **OLS regression with Newey-West standard errors** ([Notebook 1, Section 7](01_Statistical_Foundations.ipynb))
- **Autocorrelation and volatility clustering diagnostics** ([Notebook 2, Sections 3-4](02_Time_Series_Foundations.ipynb))
- **Factor regression mechanics and interpretation** ([Notebook 3](03_Fama_French_3Factor.ipynb))
- **The FF5 + Momentum model and its interpretation** ([Notebook 4, Sections 1-2](04_Advanced_Factor_Models.ipynb))
- **The DFA/Avantis implementation philosophy** ([Notebook 4, Section 6](04_Advanced_Factor_Models.ipynb))

---

## Section 1: The Dimensional and Avantis Investment Philosophies

### 1.1 Dimensional Fund Advisors — The Original Factor Firm

Dimensional was founded in 1981 by David Booth and Rex Sinquefield, with direct guidance from Eugene Fama and Kenneth French (who serve as consultants and board members). Having crossed $1 trillion USD in AUM, Dimensional is the largest firm built entirely on academic factor research.

**Core Investment Philosophy:**
- Markets are broadly efficient — don't try to pick stocks or time the market
- Academic research identifies **reliable dimensions of higher expected returns**: market, size, value, and profitability
- **Systematically tilt** portfolios toward small-cap, value, and profitable stocks
- Use **patient, low-cost trading** — don't track an index mechanically, but trade opportunistically to minimise costs
- **Broad diversification** within the tilt — hold thousands of stocks, not concentrated bets

**The Integrated Approach (from Wei Dai, Rational Reminder Episode 306):**

As discussed in [Notebook 4, Section 6.4](04_Advanced_Factor_Models.ipynb), Dimensional's key innovation is the **integrated multi-factor portfolio**. Rather than buying separate value, size, and profitability funds (the "combination" approach) or adding a concentrated factor satellite to a core holding, DFA does a **three-way simultaneous sort** on size, value, and profitability:

- Start with the entire market
- Score every stock on all three dimensions at once
- Gradually shift weight toward stocks with higher expected returns across *all* dimensions
- Use a **multiplier system**: each stock's weight = market-cap weight x multiplier (e.g., 2x for high-expected-return segments, 0.5x for low)

This avoids the problem identified by Novy-Marx ([Notebook 4, Section 6.9](04_Advanced_Factor_Models.ipynb)): buying a value fund and a profitability fund separately can **cancel out** factor exposures, because value stocks tend to score low on profitability and vice versa.

**Key Implementation Details:**
- **No hard rebalancing dates** — daily rebalancing ranges instead of quarterly index reconstitution
- **Momentum as an exclusion screen** — delay buying stocks with negative momentum (avoids "catching falling knives")
- **Reversal screens** — based on Dai & Novy-Marx (2024), delay trading when short-term reversals suggest adverse selection
- **8,000–12,000 stocks** per portfolio — extreme diversification to capture Bessembinder's insight that only 4% of stocks drive all wealth creation ([Notebook 4, Section 6.5](04_Advanced_Factor_Models.ipynb))

**Dimensional UCITS ETFs:**
Dimensional launched UCITS-compliant ETFs for European investors, including DEGC (Global Core Equity) and DEGT (Global Targeted Value). However, these launched only in late 2025, making their data history too short for meaningful statistical analysis at this time.

### 1.2 Avantis Investors — The Scientific Approach

Avantis was founded in 2019 by Eduardo Repetto (former CIO of Dimensional) and a team of ex-DFA researchers. Their approach builds on the Dimensional foundation but introduces several refinements.

**The "Scientific Approach to Investing"**

Avantis' investment philosophy, as outlined in their institutional research documents, rests on three pillars:

**Pillar 1: Markets Are Broadly Efficient, But Expected Returns Vary**
- Security prices reflect available information quickly and accurately *most of the time*
- However, expected returns are **not equal across all stocks** — some characteristics (value, profitability) are associated with higher expected returns
- This is not a contradiction: efficient markets can still offer different expected returns as compensation for different levels of risk

**Pillar 2: Identify Reliable Drivers of Expected Returns**
- Use economic theory and empirical evidence to identify characteristics that predict cross-sectional return differences
- Focus on characteristics that are: (a) persistent across time, (b) pervasive across markets, (c) robust to different measurement approaches, (d) grounded in economic intuition
- Avantis focuses on two primary signals:
  - **Valuation ratios** (book-to-market, earnings yield) — the value dimension
  - **Cash-flow-based profitability** — the profitability dimension
- These are combined into a single **composite expected return** score for each stock

**Pillar 3: Thoughtful Portfolio Construction Minimises Costs**
- The gap between theoretical factor returns and realisable returns is large (as Chen & Velikov documented — [Notebook 4, Section 7](04_Advanced_Factor_Models.ipynb))
- Avantis explicitly designs portfolios to **minimise the implementation shortfall**:
  - Broad diversification (thousands of holdings)
  - Integrated multi-factor sorting (not separate factor sleeves)
  - Patient, spread-sensitive trading
  - Momentum and reversal signals used to **time trades**, not as standalone factors

**Key Differences from Dimensional:**

| Dimension | Dimensional | Avantis |
|-----------|-------------|---------|
| **Sorting method** | Independent sorts on size × value × profitability, then multiplier weights | Composite expected-return score combining value + profitability |
| **Philosophy** | Three-way sort with gradual tilts | Rank all stocks by expected return, overweight the top |
| **Concentration** | Very broad (8,000–12,000 stocks) | Somewhat more concentrated (still thousands of stocks) |
| **Distribution** | Historically advisor-only; now also ETFs | ETF-first from launch |
| **Momentum use** | Exclusion screen only (avoid negative momentum) | Timing signal for trade execution |
| **Fee structure** | Competitive (0.15–0.35%) | Very competitive (0.15–0.25%) |

**The Avantis UCITS ETFs We'll Analyse:**

| Ticker | Full Name | Strategy | Launch (Xetra) |
|--------|-----------|----------|-----------------|
| **AVWS** | Avantis Global Small Cap Value UCITS ETF | Small-cap stocks screened for value + profitability worldwide | October 2024 |
| **AVWC** | Avantis Global Equity UCITS ETF | All-cap global equity with value + profitability tilts | October 2024 |
| **AVEM** | Avantis Emerging Markets Equity UCITS ETF | EM equity with value + profitability tilts | December 2024 |

**What we expect to find in factor regressions:**
- **AVWS**: Positive SMB (small-cap), positive HML (value tilt), positive RMW (profitability screen). This should look like a small-cap value fund with a profitability overlay.
- **AVWC**: Market beta close to 1, modest positive HML and RMW (broad market with gentle tilts). Should resemble a "market-plus" strategy.
- **AVEM**: Market beta near 1 for EM, with positive HML and RMW. May show different factor loadings due to EM-specific characteristics.

### 1.3 The Rational Reminder Perspective

The [Rational Reminder podcast](https://rationalreminder.ca/) (Ben Felix and Cameron Passmore) has become one of the most influential voices in evidence-based investing. Their framework for factor investing with ETFs aligns closely with what we'll analyse:

1. **Core market exposure** supplemented by **factor tilts** (especially small-cap value + profitability)
2. **Global diversification** across regions
3. **Systematic rebalancing** with discipline through drawdowns
4. **Fee awareness** — even small differences compound over decades

The portfolio we construct in Section 6 (50% AVWS / 40% AVWC / 10% AVEM) represents a **globally diversified, factor-tilted equity allocation** — exactly the type of portfolio this research tradition recommends.

### 1.4 How This Connects to the Academic Research

The academic threads from [Notebook 4](04_Advanced_Factor_Models.ipynb) converge in these products:

| Academic Finding | How DFA/Avantis Implement It |
|-----------------|------------------------------|
| Value + profitability interaction (Novy-Marx, 2013) | Integrated multi-factor sort — no cancellation of exposures |
| Transaction cost erosion (Chen & Velikov, 2023) | Patient trading, reversal screens, no forced rebalancing dates |
| Momentum crash risk (Daniel & Moskowitz, 2016) | Momentum as exclusion/timing screen, not as a held position |
| Bessembinder's skewness (2018) | Extreme diversification (thousands of holdings) |
| Wei Dai's 720 timing strategies (2023) | No premium timing — constant exposure to return dimensions |
| Adaptive Markets (Lo, 2017; Chen, 2024) | Continuous research cycle; willingness to update methodology |

The question we can begin to answer with data: **do the factor exposures we observe in these ETFs match what the firms claim?**

In [ ]:
# ============================================================================
# Setup: Import Libraries
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox
import yfinance as yf
import urllib.request
import zipfile
import tempfile
import os
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('default')
sns.set_palette("husl")
np.random.seed(42)

print("Libraries imported successfully!")

---

## Section 2: Data — Avantis UCITS ETFs and Fama-French Factors

We need two datasets:

1. **Daily ETF prices** from Xetra (via Yahoo Finance) for AVWS, AVWC, and AVEM
2. **Daily US Fama-French 5 factors + Momentum** from Kenneth French's data library

Since these ETFs invest *globally* (not just in US stocks), we would ideally use the **Global** factor data. However, the **Global daily factors on French's website are only available through mid-2019** — well before these ETFs launched. We therefore use the **US daily factors**, which are updated through the present. Because US equities represent ~60% of global market capitalisation and US factors are highly correlated with their global counterparts, this is a reasonable approximation — particularly for estimating factor *exposures* (betas), which is our primary goal. Alpha estimates should be interpreted with additional caution given this proxy.

> **Currency note:** The ETFs trade in EUR on Xetra, while the Fama-French factors are computed in USD. We convert ETF returns to USD using the EUR/USD exchange rate. This ensures our factor loadings are not contaminated by currency movements. The betas are relatively robust to this conversion (since currency effects are largely orthogonal to factor returns), but the alpha estimate would be meaningfully affected without it.

In [ ]:
# ============================================================================
# Step 1: Download US Fama-French 5 Factors + Momentum (Daily)
# ============================================================================

def load_ff_csv_from_zip(url, skip_footer=0):
    """Download and parse a Fama-French CSV from a zip file."""
    with tempfile.TemporaryDirectory() as tmpdir:
        zip_path = os.path.join(tmpdir, 'data.zip')
        urllib.request.urlretrieve(url, zip_path)
        with zipfile.ZipFile(zip_path, 'r') as z:
            csv_name = [f for f in z.namelist() if f.endswith('.CSV') or f.endswith('.csv')][0]
            with z.open(csv_name) as f:
                lines = f.read().decode('utf-8').splitlines()
    
    # Find the header row (contains 'Mkt-RF' or 'WML' or 'Mom')
    header_idx = None
    for i, line in enumerate(lines):
        if 'Mkt-RF' in line or 'WML' in line or 'Mom' in line:
            header_idx = i
            break
    
    if header_idx is None:
        raise ValueError("Could not find header row in CSV")
    
    # Parse from header row
    from io import StringIO
    data_text = '\n'.join(lines[header_idx:])
    df = pd.read_csv(StringIO(data_text), index_col=0)
    
    # Clean: remove non-numeric rows and annual data section
    df.index = df.index.astype(str).str.strip()
    df = df[df.index.str.match(r'^\d{8}$')]  # Keep only YYYYMMDD rows
    df.index = pd.to_datetime(df.index, format='%Y%m%d')
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.dropna()
    
    return df

# Note: We use US factors because the Global daily factors on French's website 
# only extend to mid-2019, well before these UCITS ETFs launched.
# US factors are a reasonable proxy — see Section 2 discussion.

print("Loading US Fama-French 5 Factors (Daily)...")
ff5_url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip'
ff5 = load_ff_csv_from_zip(ff5_url)
ff5.columns = ff5.columns.str.strip()

# Rename Mkt-RF to Mkt_RF for consistency with earlier notebooks
ff5 = ff5.rename(columns={'Mkt-RF': 'Mkt_RF'})

print(f"  Shape: {ff5.shape}")
print(f"  Date range: {ff5.index[0].date()} to {ff5.index[-1].date()}")
print(f"  Columns: {list(ff5.columns)}")

print("\nLoading US Momentum Factor (Daily)...")
mom_url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Momentum_Factor_daily_CSV.zip'
mom = load_ff_csv_from_zip(mom_url)
mom.columns = mom.columns.str.strip()

# Rename the momentum column to UMD for consistency
mom_col = mom.columns[0]  # Usually 'Mom' or 'WML'
mom = mom.rename(columns={mom_col: 'UMD'})

print(f"  Shape: {mom.shape}")
print(f"  Date range: {mom.index[0].date()} to {mom.index[-1].date()}")

# Merge FF5 + Momentum
factors = ff5.join(mom[['UMD']], how='inner')

print(f"\nCombined factors (FF5 + Momentum): {factors.shape}")
print(f"  Columns: {list(factors.columns)}")
print(f"\nSample (last 5 rows):")
print(factors.tail())

In [ ]:
# ============================================================================
# Step 2: Download Avantis UCITS ETF Prices from Xetra + EUR/USD FX Rate
# ============================================================================

etf_tickers = {
    'AVWS.DE': 'Avantis Global Small Cap Value',
    'AVWC.DE': 'Avantis Global Equity',
    'AVEM.DE': 'Avantis Emerging Markets Equity',
}

print("="*70)
print("DOWNLOADING AVANTIS UCITS ETF DATA (XETRA)")
print("="*70)

# Download ETF prices
etf_prices = {}
for ticker, name in etf_tickers.items():
    data = yf.download(ticker, period='max', progress=False)
    # Handle multi-level columns from yfinance
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    etf_prices[ticker] = data['Close']
    print(f"\n{ticker} ({name}):")
    print(f"  Observations: {len(data)}")
    print(f"  Date range: {data.index[0].date()} to {data.index[-1].date()}")
    print(f"  First price: {data['Close'].iloc[0]:.2f} EUR")
    print(f"  Last price: {data['Close'].iloc[-1]:.2f} EUR")

# Download EUR/USD exchange rate
print("\n" + "-"*70)
print("Downloading EUR/USD exchange rate...")
fx_data = yf.download('EURUSD=X', start='2024-09-01', progress=False)
if isinstance(fx_data.columns, pd.MultiIndex):
    fx_data.columns = fx_data.columns.get_level_values(0)
eurusd = fx_data['Close']
print(f"  Observations: {len(eurusd)}")
print(f"  Date range: {eurusd.index[0].date()} to {eurusd.index[-1].date()}")

# Compute daily returns in EUR, then convert to USD
print("\n" + "="*70)
print("COMPUTING DAILY RETURNS (USD-CONVERTED)")
print("="*70)

etf_returns_usd = {}
for ticker, name in etf_tickers.items():
    prices_eur = etf_prices[ticker]
    
    # Daily EUR returns
    ret_eur = prices_eur.pct_change().dropna()
    
    # EUR/USD rate changes
    fx_aligned = eurusd.reindex(ret_eur.index, method='ffill')
    fx_ret = fx_aligned.pct_change()
    
    # Convert to USD: R_usd ≈ R_eur + R_fx + R_eur * R_fx
    # For daily returns, the cross-term is negligible
    ret_usd = ret_eur + fx_ret + ret_eur * fx_ret
    ret_usd = ret_usd.dropna()
    
    # Convert to percentage points to match FF factor data
    etf_returns_usd[ticker] = ret_usd * 100
    
    print(f"\n{ticker} ({name}):")
    print(f"  Daily return obs: {len(ret_usd)}")
    print(f"  Mean daily return: {ret_usd.mean()*100:.4f}%")
    print(f"  Daily volatility: {ret_usd.std()*100:.4f}%")
    print(f"  Ann. return (approx): {ret_usd.mean()*252*100:.2f}%")
    print(f"  Ann. volatility (approx): {ret_usd.std()*np.sqrt(252)*100:.2f}%")

In [ ]:
# ============================================================================
# Step 3: Merge ETF Returns with Factor Data
# ============================================================================

print("="*70)
print("MERGING ETF RETURNS WITH FAMA-FRENCH FACTORS")
print("="*70)

# Create aligned dataframes for each ETF
etf_dataframes = {}
for ticker, name in etf_tickers.items():
    ret = etf_returns_usd[ticker]
    
    # Merge with factors on date
    merged = pd.DataFrame({'ETF_Return': ret})
    merged = merged.join(factors, how='inner')
    
    # Compute excess return: ETF return - risk-free rate
    merged['Excess_Return'] = merged['ETF_Return'] - merged['RF']
    
    etf_dataframes[ticker] = merged
    
    print(f"\n{ticker} ({name}):")
    print(f"  Matched observations: {len(merged)}")
    if len(merged) > 0:
        print(f"  Date range: {merged.index[0].date()} to {merged.index[-1].date()}")
        print(f"  Mean excess return: {merged['Excess_Return'].mean():.4f}%/day")
    else:
        print("  WARNING: No matching dates found!")

# Summary
print("\n" + "="*70)
print("DATA SUMMARY")
print("="*70)
print(f"{'ETF':<12} {'Obs':>6} {'Start':>12} {'End':>12} {'Ann. Ret':>10} {'Ann. Vol':>10}")
print("-"*70)
for ticker, name in etf_tickers.items():
    df = etf_dataframes[ticker]
    ann_ret = df['Excess_Return'].mean() * 252
    ann_vol = df['Excess_Return'].std() * np.sqrt(252)
    print(f"{ticker:<12} {len(df):>6} {df.index[0].strftime('%Y-%m-%d'):>12} "
          f"{df.index[-1].strftime('%Y-%m-%d'):>12} {ann_ret:>9.2f}% {ann_vol:>9.2f}%")

---

## Section 3: Descriptive Analysis

Before running factor regressions, we examine the basic statistical properties of these ETFs. This connects directly to [Notebook 2](02_Time_Series_Foundations.ipynb): financial return series typically exhibit fat tails, volatility clustering, and weak autocorrelation — features that our regression approach (Newey-West SEs) is designed to handle.

In [ ]:
# ============================================================================
# Descriptive Statistics and Return Distributions
# ============================================================================

print("="*70)
print("DESCRIPTIVE STATISTICS — DAILY EXCESS RETURNS (% per day)")
print("="*70)

desc_stats = []
for ticker, name in etf_tickers.items():
    r = etf_dataframes[ticker]['Excess_Return']
    jb_stat, jb_p, skew_val, kurt_val = jarque_bera(r)
    
    # Ljung-Box test for autocorrelation (lag 5)
    lb_result = acorr_ljungbox(r, lags=[5], return_df=True)
    lb_p = lb_result['lb_pvalue'].values[0]
    
    # Ljung-Box on squared returns (volatility clustering)
    lb_sq = acorr_ljungbox(r**2, lags=[5], return_df=True)
    lb_sq_p = lb_sq['lb_pvalue'].values[0]
    
    desc_stats.append({
        'ETF': ticker.replace('.DE', ''),
        'N': len(r),
        'Mean (%)': f"{r.mean():.4f}",
        'Std (%)': f"{r.std():.4f}",
        'Skewness': f"{skew_val:.3f}",
        'Kurtosis': f"{kurt_val:.3f}",
        'JB p-value': f"{jb_p:.4f}",
        'LB(5) p': f"{lb_p:.4f}",
        'LB²(5) p': f"{lb_sq_p:.4f}",
        'Min (%)': f"{r.min():.2f}",
        'Max (%)': f"{r.max():.2f}",
    })

desc_df = pd.DataFrame(desc_stats).set_index('ETF')
print(desc_df.to_string())

print("""
INTERPRETATION:
━━━━━━━━━━━━━━
• JB p-value < 0.05 → Returns are NOT normally distributed (expected for 
  financial data, as discussed in Notebook 2, Section 1)
  
• LB(5) p-value: Tests for autocorrelation in returns. 
  p < 0.05 → significant autocorrelation detected

• LB²(5) p-value: Tests for autocorrelation in SQUARED returns 
  (volatility clustering). p < 0.05 → ARCH effects present
  (as expected from Notebook 2, Section 4)

→ These diagnostics confirm that Newey-West standard errors are appropriate
  for our factor regressions (they account for both autocorrelation and 
  heteroskedasticity).
""")

# ============================================================================
# Visualisation: Return Distributions and Cumulative Returns
# ============================================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, (ticker, name) in enumerate(etf_tickers.items()):
    r = etf_dataframes[ticker]['Excess_Return']
    short_name = ticker.replace('.DE', '')
    
    # Top row: Return histograms with normal overlay
    ax = axes[0, i]
    ax.hist(r, bins=50, density=True, alpha=0.7, color='steelblue', 
            edgecolor='white', linewidth=0.5)
    x = np.linspace(r.min(), r.max(), 100)
    ax.plot(x, stats.norm.pdf(x, r.mean(), r.std()), 'r-', linewidth=2,
            label='Normal')
    ax.set_title(f'{short_name}: Daily Excess Returns', fontweight='bold')
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Density')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # Bottom row: Cumulative returns
    ax = axes[1, i]
    cum_ret = (1 + r / 100).cumprod()
    ax.plot(cum_ret.index, cum_ret, color='darkblue', linewidth=1.5)
    ax.axhline(1, color='red', linestyle='--', linewidth=0.8, alpha=0.5)
    ax.set_title(f'{short_name}: Cumulative Growth of $1', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value ($)')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=30)

plt.suptitle('Avantis UCITS ETFs — Return Distributions and Cumulative Performance',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('avantis_descriptive.png', dpi=100, bbox_inches='tight')
plt.show()

print("""
TOP ROW: Return distributions. Compare the histogram to the red normal curve.
Fat tails (excess kurtosis) are visible — returns have more extreme observations 
than a normal distribution would predict. This motivates robust standard errors.

BOTTOM ROW: Cumulative returns since inception. Remember: this is a very short 
period (~17 months). Short-term performance is dominated by market movements and 
luck, not factor premia (which require 10-20 year horizons per Wei Dai's research).
""")

---

## Section 4: Factor Regression Analysis

This is the core analytical section. We regress each ETF's daily excess returns on the **FF5 + Momentum (6-factor) model**:

$$R_i(t) - R_f(t) = \alpha_i + \beta_{MKT} (R_m - R_f)(t) + \beta_{SMB} \cdot SMB(t) + \beta_{HML} \cdot HML(t) + \beta_{RMW} \cdot RMW(t) + \beta_{CMA} \cdot CMA(t) + \beta_{UMD} \cdot UMD(t) + \epsilon_i(t)$$

We use **Newey-West standard errors** with automatic lag selection, as justified in [Notebook 1, Section 7](01_Statistical_Foundations.ipynb). This accounts for both autocorrelation and heteroskedasticity in the residuals — the same issues our descriptive analysis just confirmed.

For each ETF, we run both a **3-factor** and a **6-factor** regression to see how the additional factors change the picture — mirroring the progression in [Notebook 4, Section 1](04_Advanced_Factor_Models.ipynb).

In [ ]:
# ============================================================================
# Factor Regression Helper Function
# ============================================================================

def run_factor_regression(etf_data, factor_cols, ticker_name, nw_lags=None):
    """
    Run OLS regression with Newey-West standard errors.
    
    Parameters
    ----------
    etf_data : DataFrame with 'Excess_Return' and factor columns
    factor_cols : list of factor column names
    ticker_name : string for display
    nw_lags : int or None (auto = int(4*(T/100)^(2/9)))
    
    Returns
    -------
    statsmodels RegressionResults
    """
    y = etf_data['Excess_Return'].values
    X = sm.add_constant(etf_data[factor_cols].values)
    
    # Automatic Newey-West lag selection (Andrews, 1991 rule of thumb)
    T = len(y)
    if nw_lags is None:
        nw_lags = int(np.ceil(4 * (T / 100) ** (2/9)))
    
    model = sm.OLS(y, X)
    results = model.fit(cov_type='HAC', cov_kwds={'maxlags': nw_lags})
    
    return results, nw_lags

def print_regression_results(results, factor_cols, ticker_name, nw_lags):
    """Pretty-print regression results."""
    param_names = ['Alpha (α)'] + factor_cols
    
    print(f"\n{'='*65}")
    print(f"  {ticker_name}")
    print(f"  Newey-West lags: {nw_lags} | N = {int(results.nobs)} | R² = {results.rsquared:.4f}")
    print(f"{'='*65}")
    print(f"{'Factor':<14} {'Coeff':>10} {'Std Err':>10} {'t-stat':>10} {'p-value':>10} {'Sig':>5}")
    print(f"{'-'*65}")
    
    for i, name in enumerate(param_names):
        coef = results.params[i]
        se = results.bse[i]
        t = results.tvalues[i]
        p = results.pvalues[i]
        sig = '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''
        print(f"{name:<14} {coef:>10.4f} {se:>10.4f} {t:>10.3f} {p:>10.4f} {sig:>5}")
    
    # Annualise alpha (daily → annual, approx)
    alpha_daily = results.params[0]
    alpha_annual = alpha_daily * 252
    print(f"\n  Annualised alpha ≈ {alpha_annual:.2f}% (= {alpha_daily:.4f}% × 252)")
    print(f"  Residual std (daily) = {np.sqrt(results.mse_resid):.4f}%")

print("Regression helper functions defined.")

In [ ]:
# ============================================================================
# Run FF3 and FF5+Momentum Regressions for Each ETF
# ============================================================================

ff3_cols = ['Mkt_RF', 'SMB', 'HML']
ff6_cols = ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']

results_3f = {}
results_6f = {}

print("█"*70)
print("  FAMA-FRENCH 3-FACTOR REGRESSIONS (GLOBAL FACTORS)")
print("█"*70)

for ticker, name in etf_tickers.items():
    df = etf_dataframes[ticker]
    res, lags = run_factor_regression(df, ff3_cols, f"{ticker} — {name}")
    results_3f[ticker] = res
    print_regression_results(res, ff3_cols, f"{ticker} — {name} [3-Factor]", lags)

print("\n\n" + "█"*70)
print("  FAMA-FRENCH 5-FACTOR + MOMENTUM (6-FACTOR) REGRESSIONS")
print("█"*70)

for ticker, name in etf_tickers.items():
    df = etf_dataframes[ticker]
    res, lags = run_factor_regression(df, ff6_cols, f"{ticker} — {name}")
    results_6f[ticker] = res
    print_regression_results(res, ff6_cols, f"{ticker} — {name} [6-Factor]", lags)

print("\n\n" + "="*70)
print("COMPARING 3-FACTOR VS 6-FACTOR: WHAT CHANGES?")
print("="*70)
print(f"\n{'ETF':<12} {'R² (3F)':>10} {'R² (6F)':>10} {'ΔR²':>8} {'α (3F)':>10} {'α (6F)':>10}")
print("-"*60)
for ticker in etf_tickers:
    r3 = results_3f[ticker]
    r6 = results_6f[ticker]
    short = ticker.replace('.DE', '')
    print(f"{short:<12} {r3.rsquared:>10.4f} {r6.rsquared:>10.4f} "
          f"{r6.rsquared - r3.rsquared:>8.4f} "
          f"{r3.params[0]*252:>9.2f}% {r6.params[0]*252:>9.2f}%")

print("""
WHAT TO LOOK FOR:
━━━━━━━━━━━━━━━━
• AVWS (Small Cap Value): Should show positive SMB (small-cap tilt), 
  positive HML (value tilt), and positive RMW (profitability screen).
  This is the purest factor-tilted fund in our set.

• AVWC (Global Equity): Market beta ≈ 1, with modest positive HML and 
  RMW. The tilts should be gentler than AVWS since this is a broad fund.

• AVEM (Emerging Markets): Market beta should be significant. Factor 
  loadings may differ because EM stocks have different characteristics.

• Alpha: With only ~17 months of data, alpha estimates are very noisy.
  Don't over-interpret whether alpha is positive or negative.

• R² increase from 3F to 6F: If adding RMW, CMA, UMD substantially 
  increases R², it means these factors capture real variation in the 
  ETF's returns — consistent with the ETFs having multi-factor tilts.
""")

In [ ]:
# ============================================================================
# Residual Diagnostics — Validating Our Regression Assumptions
# ============================================================================
# Following the diagnostic approach from Notebook 3, Section 5

print("="*70)
print("RESIDUAL DIAGNOSTICS (6-FACTOR MODEL)")
print("="*70)
print("Checking the assumptions underlying our OLS + Newey-West regressions.\n")

fig, axes = plt.subplots(3, 3, figsize=(18, 14))

for i, (ticker, name) in enumerate(etf_tickers.items()):
    res = results_6f[ticker]
    resid = res.resid
    short = ticker.replace('.DE', '')
    
    # Column 1: Residual time series
    ax = axes[i, 0]
    dates = etf_dataframes[ticker].index
    ax.plot(dates, resid, linewidth=0.6, color='steelblue', alpha=0.8)
    ax.axhline(0, color='red', linestyle='--', linewidth=0.8)
    ax.set_title(f'{short}: Residuals Over Time', fontweight='bold', fontsize=10)
    ax.set_ylabel('Residual (%)')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=30)
    
    # Column 2: Residual ACF
    ax = axes[i, 1]
    from statsmodels.graphics.tsaplots import plot_acf
    plot_acf(resid, ax=ax, lags=20, alpha=0.05)
    ax.set_title(f'{short}: Residual ACF', fontweight='bold', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Column 3: QQ Plot
    ax = axes[i, 2]
    stats.probplot(resid, dist='norm', plot=ax)
    ax.set_title(f'{short}: QQ Plot', fontweight='bold', fontsize=10)
    ax.grid(True, alpha=0.3)

plt.suptitle('6-Factor Model — Residual Diagnostics',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('avantis_diagnostics.png', dpi=100, bbox_inches='tight')
plt.show()

# Formal tests
print("\nFORMAL DIAGNOSTIC TESTS:")
print(f"{'ETF':<10} {'JB p-val':>10} {'LB(5) p':>10} {'LB²(5) p':>10} {'Resid kurt':>12}")
print("-"*55)
for ticker in etf_tickers:
    resid = results_6f[ticker].resid
    jb_stat, jb_p, skew, kurt = jarque_bera(resid)
    lb = acorr_ljungbox(resid, lags=[5], return_df=True)['lb_pvalue'].values[0]
    lb_sq = acorr_ljungbox(resid**2, lags=[5], return_df=True)['lb_pvalue'].values[0]
    short = ticker.replace('.DE', '')
    print(f"{short:<10} {jb_p:>10.4f} {lb:>10.4f} {lb_sq:>10.4f} {kurt:>12.2f}")

print("""
INTERPRETATION:
━━━━━━━━━━━━━━
LEFT: Residuals should look like white noise — no visible patterns or 
clustering. Any remaining volatility clustering means the 6-factor model 
doesn't fully capture all systematic return variation.

MIDDLE: ACF of residuals. Bars outside the blue confidence band indicate 
significant autocorrelation. Newey-West SEs handle this, but strong 
autocorrelation would suggest model misspecification.

RIGHT: QQ plot. Deviations from the diagonal in the tails indicate fat tails 
(excess kurtosis) — normal for financial data. This affects confidence 
interval coverage but not the consistency of OLS estimates 
(Notebook 1, Section 5).

LB²(5) p-value: If significant, ARCH effects remain in the residuals — 
confirming that Newey-West (HAC) standard errors are essential.
""")

---

## Section 5: Comparative Factor Profiles

Now we compare the factor exposures across all three ETFs side by side. This reveals how each fund implements Avantis' "scientific approach" differently depending on its investment universe (global small-cap value vs. broad global equity vs. emerging markets).

In [ ]:
# ============================================================================
# Comparative Factor Exposure Chart
# ============================================================================

factor_names = ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']
etf_short_names = [t.replace('.DE', '') for t in etf_tickers.keys()]
colors = ['#2196F3', '#4CAF50', '#FF9800']  # Blue, Green, Orange

# Collect coefficients and significance
coef_matrix = np.zeros((len(etf_tickers), len(factor_names)))
sig_matrix = np.zeros((len(etf_tickers), len(factor_names)))
se_matrix = np.zeros((len(etf_tickers), len(factor_names)))

for i, ticker in enumerate(etf_tickers.keys()):
    res = results_6f[ticker]
    for j, factor in enumerate(factor_names):
        coef_matrix[i, j] = res.params[j + 1]  # +1 to skip constant
        sig_matrix[i, j] = res.pvalues[j + 1]
        se_matrix[i, j] = res.bse[j + 1]

# Plot 1: Side-by-side bar chart of factor loadings
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Panel A: Factor Betas
ax = axes[0]
x = np.arange(len(factor_names))
width = 0.25
for i, (etf_name, color) in enumerate(zip(etf_short_names, colors)):
    bars = ax.bar(x + i * width - width, coef_matrix[i], width, 
                  label=etf_name, color=color, alpha=0.85, edgecolor='white')
    # Add significance markers
    for j, bar in enumerate(bars):
        if sig_matrix[i, j] < 0.01:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    '***', ha='center', fontsize=8, fontweight='bold')
        elif sig_matrix[i, j] < 0.05:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    '**', ha='center', fontsize=8, fontweight='bold')
        elif sig_matrix[i, j] < 0.10:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    '*', ha='center', fontsize=8)

ax.set_xticks(x)
ax.set_xticklabels(factor_names, fontsize=11)
ax.set_ylabel('Factor Loading (β)', fontsize=12)
ax.set_title('Panel A: Factor Exposures (6-Factor Model)', fontweight='bold',
             fontsize=13)
ax.axhline(0, color='black', linewidth=0.8)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Panel B: Alpha comparison (annualised)
ax = axes[1]
alphas = [results_6f[t].params[0] * 252 for t in etf_tickers.keys()]
alpha_ses = [results_6f[t].bse[0] * 252 for t in etf_tickers.keys()]  # approx
alpha_sigs = [results_6f[t].pvalues[0] for t in etf_tickers.keys()]

bars = ax.bar(etf_short_names, alphas, color=colors, alpha=0.85, 
              edgecolor='white', width=0.5)
ax.errorbar(etf_short_names, alphas, yerr=[1.96*se for se in alpha_ses],
            fmt='none', color='black', capsize=5, linewidth=1.5)
ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.set_ylabel('Annualised Alpha (%)', fontsize=12)
ax.set_title('Panel B: Jensen\'s Alpha (6-Factor, Annualised)', 
             fontweight='bold', fontsize=13)
ax.grid(True, alpha=0.3, axis='y')

# Add p-values
for i, (bar, p) in enumerate(zip(bars, alpha_sigs)):
    label = f'p={p:.3f}'
    ax.text(bar.get_x() + bar.get_width()/2, 
            bar.get_height() + alpha_ses[i] * 2.2,
            label, ha='center', fontsize=10)

plt.suptitle('Avantis UCITS ETFs — Comparative Factor Analysis',
             fontsize=15, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig('avantis_factor_comparison.png', dpi=100, bbox_inches='tight')
plt.show()

# Summary table
print("\n" + "="*70)
print("FACTOR EXPOSURE SUMMARY (6-FACTOR MODEL)")
print("="*70)
print(f"\n{'Factor':<10}", end='')
for name in etf_short_names:
    print(f"  {name:>14}", end='')
print()
print("-"*55)
for j, factor in enumerate(factor_names):
    print(f"{factor:<10}", end='')
    for i in range(len(etf_tickers)):
        coef = coef_matrix[i, j]
        p = sig_matrix[i, j]
        sig = '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''
        print(f"  {coef:>10.4f}{sig:>4}", end='')
    print()

print(f"\n{'R²':<10}", end='')
for ticker in etf_tickers:
    print(f"  {results_6f[ticker].rsquared:>14.4f}", end='')
print()

print("""
INTERPRETATION GUIDE:
━━━━━━━━━━━━━━━━━━━━
                            Expected for        Expected for        Expected for
Factor      Meaning         AVWS (SCV)          AVWC (Global)       AVEM (EM)
─────────────────────────────────────────────────────────────────────────────────
Mkt_RF      Market risk     ~0.8–1.0            ~1.0                ~0.8–1.0
SMB         Size            Positive (small!)    Near zero           Near zero
HML         Value           Positive (value!)    Mild positive       Mild positive
RMW         Profitability   Positive (screen)    Mild positive       Positive
CMA         Investment      Ambiguous            Near zero           Ambiguous
UMD         Momentum        Near zero/positive   Near zero           Near zero

*** p < 0.01, ** p < 0.05, * p < 0.10 (Newey-West standard errors)
""")

---

## Section 6: Portfolio Construction — 50% AVWS / 40% AVWC / 10% AVEM

Now we construct a combined portfolio and analyse its characteristics. This allocation represents a **globally diversified, factor-tilted equity portfolio**:

- **50% AVWS** — A heavy tilt toward global small-cap value (the strongest documented factor premia)
- **40% AVWC** — Broad global equity with gentle factor tilts (core holding, provides diversification)
- **10% AVEM** — Emerging markets exposure (geographic diversification, potentially different factor dynamics)

This type of allocation aligns with the Rational Reminder framework: a factor-tilted core supplemented by a meaningful small-cap value overweight, with some EM diversification.

### Why These Weights?

The 50/40/10 split reflects several considerations:
1. **Academic evidence is strongest for small-cap value** — Wei Dai's research shows small-cap value has the highest expected premium, so a large AVWS allocation targets this
2. **Diversification requires breadth** — AVWC provides broad market exposure and reduces tracking error vs. a pure small-cap value bet (Bessembinder's skewness argument)
3. **EM adds geographic diversity** — Factor premia have been documented globally (Asness, Moskowitz & Pedersen, 2013), but EM markets may behave differently

> **Important:** We can only construct this portfolio from the date when all three ETFs have available data (the latest inception date). Since AVEM launched in December 2024, our combined portfolio history is limited to approximately 15 months.

In [ ]:
# ============================================================================
# 6.1 — Build the Portfolio
# ============================================================================

weights = {'AVWS.DE': 0.50, 'AVWC.DE': 0.40, 'AVEM.DE': 0.10}

# Find common date range where all three ETFs have data
common_dates = etf_dataframes['AVWS.DE'].index
for ticker in ['AVWC.DE', 'AVEM.DE']:
    common_dates = common_dates.intersection(etf_dataframes[ticker].index)

print(f"Common dates across all three ETFs: {len(common_dates)}")
print(f"Date range: {common_dates[0].date()} to {common_dates[-1].date()}")
print()

# Build a combined dataframe on common dates
port_data = pd.DataFrame(index=common_dates)
for ticker in etf_tickers:
    short = ticker.replace('.DE', '')
    port_data[f'{short}_excess'] = etf_dataframes[ticker].loc[common_dates, 'Excess_Return']
    port_data[f'{short}_ret'] = etf_dataframes[ticker].loc[common_dates, 'ETF_Return']

# Add factor columns from any ETF's dataframe (they're identical)
factor_cols_all = ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'RF']
for col in factor_cols_all:
    port_data[col] = etf_dataframes['AVWS.DE'].loc[common_dates, col]

# Compute portfolio returns (weighted average)
port_data['Port_ret'] = (weights['AVWS.DE'] * port_data['AVWS_ret']
                       + weights['AVWC.DE'] * port_data['AVWC_ret']
                       + weights['AVEM.DE'] * port_data['AVEM_ret'])

port_data['Port_excess'] = (weights['AVWS.DE'] * port_data['AVWS_excess']
                          + weights['AVWC.DE'] * port_data['AVWC_excess']
                          + weights['AVEM.DE'] * port_data['AVEM_excess'])

# ---- Summary Statistics ----
print("=" * 65)
print("Portfolio Summary Statistics (daily, USD-converted, % units)")
print("=" * 65)

port_ret = port_data['Port_ret']
ann_factor = 252

summary = {
    'Mean daily return (%)':           port_ret.mean(),
    'Std. dev. daily (%)':             port_ret.std(),
    'Annualised return (%)':           port_ret.mean() * ann_factor,
    'Annualised volatility (%)':       port_ret.std() * np.sqrt(ann_factor),
    'Sharpe ratio (annualised)':       (port_ret.mean() * ann_factor) / (port_ret.std() * np.sqrt(ann_factor)),
    'Skewness':                        port_ret.skew(),
    'Excess kurtosis':                 port_ret.kurtosis(),
    'Minimum daily return (%)':        port_ret.min(),
    'Maximum daily return (%)':        port_ret.max(),
}

for k, v in summary.items():
    print(f"  {k:<35s}  {v:>8.4f}")

# ---- Compare portfolio vs. individual ETFs ----
print("\n" + "=" * 65)
print("Risk-Return Comparison: Portfolio vs. Individual ETFs")
print("=" * 65)
print(f"  {'':20s}  {'Ann. Ret.':>10s}  {'Ann. Vol.':>10s}  {'Sharpe':>8s}")
print("  " + "-" * 52)

for ticker, name in etf_tickers.items():
    short = ticker.replace('.DE', '')
    r = port_data[f'{short}_ret']
    ann_r = r.mean() * ann_factor
    ann_v = r.std() * np.sqrt(ann_factor)
    sr = ann_r / ann_v if ann_v > 0 else 0
    print(f"  {short:20s}  {ann_r:>9.2f}%  {ann_v:>9.2f}%  {sr:>8.4f}")

ann_r = port_ret.mean() * ann_factor
ann_v = port_ret.std() * np.sqrt(ann_factor)
sr = summary['Sharpe ratio (annualised)']
print(f"  {'Portfolio (50/40/10)':20s}  {ann_r:>9.2f}%  {ann_v:>9.2f}%  {sr:>8.4f}")
print()
print("  Note: short-sample estimates; interpret with caution.")

In [ ]:
# ============================================================================
# 6.2 — Cumulative Returns and Drawdown Analysis
# ============================================================================

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True,
                         gridspec_kw={'height_ratios': [2, 1]})

# Panel A: Cumulative returns  (returns are in % units, so divide by 100)
cum_port = (1 + port_data['Port_ret'] / 100).cumprod()
for short_name, colour in [('AVWS', '#1f77b4'), ('AVWC', '#ff7f0e'), ('AVEM', '#2ca02c')]:
    cum_etf = (1 + port_data[f'{short_name}_ret'] / 100).cumprod()
    axes[0].plot(cum_etf.index, cum_etf.values, label=short_name, alpha=0.6, color=colour)

axes[0].plot(cum_port.index, cum_port.values, label='Portfolio (50/40/10)',
             color='black', linewidth=2.5)
axes[0].set_ylabel('Cumulative Return ($1 invested)')
axes[0].legend(loc='upper left')
axes[0].set_title('Panel A: Cumulative Returns')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(1, color='grey', linestyle='--', alpha=0.5)

# Panel B: Portfolio drawdown
cum_max = cum_port.cummax()
drawdown = (cum_port - cum_max) / cum_max * 100
axes[1].fill_between(drawdown.index, drawdown.values, 0, color='red', alpha=0.3)
axes[1].plot(drawdown.index, drawdown.values, color='red', linewidth=0.8)
axes[1].set_ylabel('Drawdown (%)')
axes[1].set_title('Panel B: Portfolio Drawdown')
axes[1].grid(True, alpha=0.3)

# Annotate maximum drawdown
max_dd = drawdown.min()
max_dd_date = drawdown.idxmin()
axes[1].annotate(f'Max DD: {max_dd:.1f}%',
                 xy=(max_dd_date, max_dd),
                 xytext=(max_dd_date + pd.Timedelta(days=30), max_dd + 2),
                 arrowprops=dict(arrowstyle='->', color='darkred'),
                 fontsize=10, color='darkred', fontweight='bold')

fig.suptitle('Portfolio Performance: 50% AVWS / 40% AVWC / 10% AVEM',
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

print(f"\n  Maximum drawdown: {max_dd:.2f}% on {max_dd_date.date()}")
print(f"  Days from peak to trough: "
      f"{(max_dd_date - cum_port[:max_dd_date].idxmax()).days}")

---

## Section 7: Portfolio Factor Analysis

Having analysed each ETF individually, we now run the same regressions on the **combined portfolio**. This serves two purposes:

1. **Verification** — Portfolio-level betas should approximately equal the weighted average of individual ETF betas (a direct consequence of the linearity of OLS).
2. **Holistic view** — The portfolio alpha, $R^2$, and residual diagnostics tell us about the *overall* factor profile of our allocation.

### The Linearity Property

Because OLS is a linear operator, if the portfolio return is:
$$
R_p = w_1 R_1 + w_2 R_2 + w_3 R_3
$$

then the portfolio beta on any factor $j$ satisfies:
$$
\hat{\beta}_{p,j} = w_1 \hat{\beta}_{1,j} + w_2 \hat{\beta}_{2,j} + w_3 \hat{\beta}_{3,j}
$$

This holds exactly when we use the same sample and factors for all regressions. We verify this property below.

In [ ]:
# ============================================================================
# 7.1 — Portfolio Factor Regressions
# ============================================================================

# Build a temporary DataFrame for the portfolio regression
port_reg_data = port_data[['Port_excess'] + ff6_cols].copy()
port_reg_data = port_reg_data.rename(columns={'Port_excess': 'Excess_Return'})

# FF3 regression on portfolio
res_port_3f, lags_3f = run_factor_regression(port_reg_data, ff3_cols, 
                                              'Portfolio (50/40/10)')
print("=" * 70)
print("PORTFOLIO (50% AVWS / 40% AVWC / 10% AVEM) — Fama-French 3-Factor")
print("=" * 70)
print_regression_results(res_port_3f, ff3_cols, 'Portfolio [3-Factor]', lags_3f)

print("\n")

# FF6 regression on portfolio
res_port_6f, lags_6f = run_factor_regression(port_reg_data, ff6_cols, 
                                              'Portfolio (50/40/10)')
print("=" * 70)
print("PORTFOLIO (50% AVWS / 40% AVWC / 10% AVEM) — FF5 + Momentum")
print("=" * 70)
print_regression_results(res_port_6f, ff6_cols, 'Portfolio [6-Factor]', lags_6f)

In [ ]:
# ============================================================================
# 7.2 — Verify Linearity: Weighted-Average Betas vs. Portfolio Betas
# ============================================================================

# Re-run individual ETF regressions on the SAME common sample
etf_results_common = {}
weight_map = {'AVWS.DE': 0.50, 'AVWC.DE': 0.40, 'AVEM.DE': 0.10}

for ticker in etf_tickers:
    short = ticker.replace('.DE', '')
    # Build a temp DataFrame with the common-sample data
    temp_df = port_data[[f'{short}_excess'] + ff6_cols].copy()
    temp_df = temp_df.rename(columns={f'{short}_excess': 'Excess_Return'})
    res_i, _ = run_factor_regression(temp_df, ff6_cols, ticker)
    etf_results_common[ticker] = res_i

# Compute weighted-average betas
param_names = ['Alpha'] + ff6_cols
print("=" * 75)
print("Linearity Verification: Portfolio β  vs.  Σ wᵢ · βᵢ  (FF6 model)")
print("=" * 75)
print(f"  {'Factor':<10s}  {'Portfolio β':>12s}  {'Weighted Avg':>12s}  {'Difference':>12s}")
print("  " + "-" * 50)

for j, factor in enumerate(param_names):
    beta_port = res_port_6f.params[j]
    beta_wavg = sum(weight_map[t] * etf_results_common[t].params[j]
                    for t in etf_tickers)
    diff = beta_port - beta_wavg
    print(f"  {factor:<10s}  {beta_port:>12.6f}  {beta_wavg:>12.6f}  {diff:>12.2e}")

print()
print("  Differences are at machine precision — linearity of OLS confirmed.")
print("  This means portfolio factor exposure is fully determined by the")
print("  individual ETF exposures and portfolio weights.")

In [ ]:
# ============================================================================
# 7.3 — Portfolio Factor Exposure Decomposition
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

factor_labels = ff6_cols  # ['Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']
etf_list = list(etf_tickers.keys())
etf_short = [t.replace('.DE', '') for t in etf_list]
colours = {'AVWS.DE': '#1f77b4', 'AVWC.DE': '#ff7f0e', 'AVEM.DE': '#2ca02c'}

# Panel A: Stacked contribution to portfolio betas
contributions = {}
for factor in factor_labels:
    j = ff6_cols.index(factor) + 1  # +1 to skip constant
    contributions[factor] = [weight_map[t] * etf_results_common[t].params[j]
                             for t in etf_list]

x = np.arange(len(factor_labels))
width = 0.5

# Stacked bars (handle positive and negative separately)
bottom_pos = np.zeros(len(factor_labels))
bottom_neg = np.zeros(len(factor_labels))

for i, ticker in enumerate(etf_list):
    short = ticker.replace('.DE', '')
    vals = np.array([contributions[f][i] for f in factor_labels])
    pos_vals = np.where(vals >= 0, vals, 0)
    neg_vals = np.where(vals < 0, vals, 0)

    axes[0].bar(x, pos_vals, width, bottom=bottom_pos, 
                label=f'{short} ({weight_map[ticker]:.0%})',
                color=colours[ticker], alpha=0.8, edgecolor='white', linewidth=0.5)
    axes[0].bar(x, neg_vals, width, bottom=bottom_neg,
                color=colours[ticker], alpha=0.8, edgecolor='white', linewidth=0.5)
    bottom_pos += pos_vals
    bottom_neg += neg_vals

# Overlay the total portfolio beta as markers
port_betas = [res_port_6f.params[ff6_cols.index(f) + 1] for f in factor_labels]
axes[0].scatter(x, port_betas, color='black', zorder=5, s=60, marker='D',
                label='Portfolio β (total)')

axes[0].set_xticks(x)
axes[0].set_xticklabels(factor_labels, rotation=45, ha='right')
axes[0].set_ylabel('Beta Coefficient')
axes[0].set_title('Panel A: Decomposition of Portfolio Factor Exposures')
axes[0].legend(loc='best', fontsize=8)
axes[0].axhline(0, color='grey', linewidth=0.5)
axes[0].grid(True, alpha=0.3, axis='y')

# Panel B: R² comparison
r2_vals = [etf_results_common[t].rsquared for t in etf_list]
r2_vals.append(res_port_6f.rsquared)
labels = etf_short + ['Portfolio']
bar_colours = [colours[t] for t in etf_list] + ['black']

bars = axes[1].bar(labels, r2_vals, color=bar_colours, alpha=0.8, edgecolor='white')
for bar, val in zip(bars, r2_vals):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                 f'{val:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

axes[1].set_ylabel('R²')
axes[1].set_title('Panel B: Model Fit (FF5 + Momentum)')
axes[1].set_ylim(0, 1.05)
axes[1].grid(True, alpha=0.3, axis='y')

fig.suptitle('Portfolio Factor Exposure Analysis', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

---

## Section 8: Limitations, Caveats, and Conclusions

### 8.1 Data Limitations

This analysis comes with important caveats that must inform interpretation:

**Short sample period.** With approximately 12–15 months of daily data, we have ~250–300 observations. While daily frequency provides *statistical power for estimating betas*, it is **insufficient for reliable alpha estimation** or performance conclusions:

- At daily frequency, the standard error of annualised alpha is typically 2–4% p.a., meaning even a "large" alpha of 1% p.a. would be statistically indistinguishable from zero
- The sample spans a single market regime — we cannot evaluate how these factor tilts perform through a full business cycle
- As emphasised throughout this series (Notebook 1, Section 3), **short samples amplify estimation error** and can produce misleading point estimates

**US factors as proxy for global exposures.** Because the Global daily factors from Ken French's data library are only available through mid-2019 (well before these UCITS ETFs launched), we use US daily factors as a proxy. Since US equities represent ~60% of global market capitalisation and US factor returns are highly correlated with their global counterparts, this is a reasonable approximation for estimating factor *exposures* (betas). However, it explains two features of our results:

- **Lower market betas:** The CAPM-predicted market beta for a globally diversified ETF regressed on US market excess returns is less than 1.0, because the US market does not perfectly capture the global market factor. The ETFs have substantial non-US exposure that is orthogonal to the US market.
- **Lower R² values:** The 6-factor model explains less variance than it would with properly matched global factors, because the residual contains both idiosyncratic risk and non-US systematic risk.

**Currency conversion noise.** Converting EUR-denominated ETF returns to USD using the EUR/USD exchange rate introduces additional noise. While this is methodologically correct for comparison with USD-denominated factors, the FX conversion adds a component to returns that is unrelated to factor exposure.

**Survivorship and incubation.** These ETFs are relatively new products. Their return histories do not reflect the long sample periods over which factor premia have been documented (typically 50+ years for US data, 30+ years for international).

### 8.2 What We *Can* Conclude

Despite these limitations, daily factor regressions allow us to draw **robust conclusions about factor exposure** (betas):

1. **AVWS delivers meaningful SMB and HML tilts** — This confirms that the fund's mandate (global small-cap value) is reflected in its actual holdings and returns, consistent with Avantis's "systematic and deliberate" approach
2. **AVWC provides broad market exposure with modest factor tilts** — Its moderate market beta and smaller factor loadings make it suitable as a core holding
3. **AVEM adds differentiated exposure** — Emerging market returns are not fully spanned by US/global factors, providing genuine diversification (as reflected in its low R²)
4. **The portfolio achieves balanced factor exposure** — The 50/40/10 allocation delivers positive SMB and HML loadings while maintaining diversification

### 8.3 Connecting Back to the Series

This notebook applies concepts from every preceding chapter:

| Concept | Source | Application Here |
|:--------|:-------|:-----------------|
| OLS regression and hypothesis testing | Notebook 1, Sections 4–5 | Factor regressions on each ETF |
| Newey-West standard errors | Notebook 1, Section 6 | HAC-robust inference throughout |
| Time series stationarity | Notebook 2, Section 2 | Using returns (stationary) not prices |
| Autocorrelation diagnostics | Notebook 2, Sections 3–4 | Ljung-Box tests on residuals |
| Fama-French 3-Factor Model | Notebook 3, Sections 3–5 | FF3 regressions as baseline |
| FF5 and Momentum extensions | Notebook 4, Sections 2–3 | Full 6-factor model |
| Rolling-window estimation | Notebook 4, Section 5 | Could be applied as data grows |
| Residual diagnostics and model evaluation | Notebooks 3–4 | ACF, Q-Q, Jarque-Bera analysis |

### 8.4 Practical Takeaways

For the factor-aware investor:

1. **Factor investing is implementable** — Products like Avantis's UCITS ETFs allow investors to harvest factor premia without building custom portfolios. The expense ratios (0.25–0.36% p.a.) represent a significant reduction from the theoretical maximum premium.

2. **Diversification across factors and geographies matters** — No single factor dominates in all periods. A portfolio that targets multiple factors (value, size, profitability, momentum) through diversified vehicles is more robust than concentrated factor bets.

3. **Patience is required** — Factor premia are long-run phenomena. The data here covers a single short period. A proper evaluation requires 10+ years of data spanning multiple market environments, as discussed in Dai (2023) and the Dimensional research.

4. **Monitor but don't overreact** — As the sample grows, these analyses should be re-run. Short-term deviations from expected factor behaviour are *normal* — persistence of factor premia is measured in decades, not months.

> **Future work:** As these ETFs accumulate longer track records, this analysis should be extended with:
> - Monthly frequency regressions (once 60+ months are available)
> - Rolling-window analysis to assess stability of factor exposures
> - Comparison with US-listed equivalents (e.g., AVUV, AVGV) that have longer histories
> - Re-estimation using Global daily factors once they are updated beyond mid-2019
> - Conditional analysis across different market regimes (bull/bear, high/low volatility)